In [1]:
import pandas as pd
import numpy as np
import mysql.connector
import datetime as dt

In [2]:
connection=mysql.connector.connect(host='localhost',user='root',database='international',password='gigaberosql')

In [3]:
df=pd.read_sql('select * from results',connection)

In [4]:
df=df[['ID','date','home_team','away_team','home_score','away_score','tournament']]

In [5]:
df=df.query('tournament=="FIFA World Cup"').reset_index(drop=True)

In [6]:
df['date']=pd.to_datetime(df['date'])
df['year']=df['date'].dt.year

In [8]:
df.tail()

,ID,date,home_team,away_team,home_score,away_score,tournament,year
895,40130,2018-07-07,Russia,Croatia,2,2,FIFA World Cup,2018
896,40131,2018-07-10,France,Belgium,1,0,FIFA World Cup,2018
897,40132,2018-07-11,Croatia,England,2,1,FIFA World Cup,2018
898,40133,2018-07-14,Belgium,England,2,0,FIFA World Cup,2018
899,40134,2018-07-15,France,Croatia,4,2,FIFA World Cup,2018


In [9]:
#df[df.columns]

df_reversed=df[['ID', 'date', 'away_team','home_team', 'away_score', 'home_score','tournament', 'year']]
t1=pd.DataFrame(np.concatenate((df.values,df_reversed.values)),columns=df.columns)

t1['matches_played']=t1.groupby(['year','home_team'])['home_team'].transform(lambda x:x.count())

In [10]:
t1

,ID,date,home_team,away_team,home_score,away_score,tournament,year,matches_played
0,1315,1930-07-13,Belgium,United States,0,3,FIFA World Cup,1930,2
1,1316,1930-07-13,France,Mexico,4,1,FIFA World Cup,1930,3
2,1317,1930-07-14,Brazil,Yugoslavia,1,2,FIFA World Cup,1930,2
3,1318,1930-07-14,Peru,Romania,1,3,FIFA World Cup,1930,2
4,1319,1930-07-15,Argentina,France,1,0,FIFA World Cup,1930,5
...,...,...,...,...,...,...,...,...,...
1795,40130,2018-07-07,Croatia,Russia,2,2,FIFA World Cup,2018,7
1796,40131,2018-07-10,Belgium,France,0,1,FIFA World Cup,2018,7
1797,40132,2018-07-11,England,Croatia,1,2,FIFA World Cup,2018,7
1798,40133,2018-07-14,England,Belgium,0,2,FIFA World Cup,2018,7


In [64]:
loser_teams=t1.sort_values(['year','home_team','date']).reset_index(drop=True)
loser_teams['rank']=loser_teams.sort_values(['date','home_team',]).groupby(['year','home_team']).cumcount()+1
loser_teams=loser_teams.query('home_score<away_score and rank==1')[['year','home_team']]
loser_teams

,year,home_team
5,1930,Belgium
7,1930,Bolivia
9,1930,Brazil
17,1930,Mexico
20,1930,Paraguay
...,...,...
1752,2018,Peru
1755,2018,Poland
1767,2018,Saudi Arabia
1776,2018,South Korea


In [70]:
winner_teams=t1.sort_values(['year','home_team','date']).query('matches_played>3').reset_index(drop=True)[['year','home_team']].drop_duplicates()
winner_teams

,year,home_team
0,1930,Argentina
5,1930,Uruguay
9,1934,Austria
13,1934,Czechoslovakia
17,1934,Germany
...,...,...
1129,2018,Russia
1134,2018,Spain
1138,2018,Sweden
1143,2018,Switzerland


In [113]:
# teams that lost first group match at world cup, but made it through the knockout stage
lucky_teams=loser_teams.merge(winner_teams)
lucky_teams

,year,home_team
0,1958,Czechoslovakia
1,1962,England
2,1962,Yugoslavia
3,1966,Hungary
4,1966,North Korea
5,1974,Argentina
6,1982,Argentina
7,1982,France
8,1982,Germany
9,1982,Russia


In [125]:
# probability of a team to reach 1/8 finals after losing first group match at world cup
'{:.2%}'.format(((len(lucky_teams))/len(loser_teams)))

'15.43%'